## Number of energy consumption
For each gaussian:
- `preprocessCUDA`: 151 MACs
For each elem:
- `radix sort`: 24 MACs
For each pixel:
- `render`: 3 + r * (1 + 256 * 14) + 2 * 3 = 9 + r * 3585
## Basic Information
width: 1200
height: 680

Mapping step 5:
tile.x, tile.y = 75, 43
num_sort = 23463167
P = 867364
mean rounds: 2.9
visible gaussians = 833384

## Metrics
Original for frames 30:

- Final Average ATE RMSE: 0.04 cm
- Average PSNR: 40.63
- Average Depth RMSE: 0.18 cm
- Average Depth L1: 0.18 cm
- Average MS-SSIM: 0.996
- Average LPIPS: 0.024

In [ ]:
tile_x, tile_y = 75, 43
block_size = 256
e = 0.5 # pj/MAC

# Mapping step 5:
sort_size = 23463167
P = 867364
visible_gaussians = 833384
mean_rounds = 2.9

preprocess = 151 * visible_gaussians
radix_sort = 24 * sort_size
render =  (3 + mean_rounds * (1 + block_size * 14) + 2 * 3) * block_size * tile_x * tile_y

print(f"Preprocess: {preprocess * e / 1e6:.2f} muj")
print(f"Radix sort: {radix_sort * e / 1e6:.2f} muj")
print(f"Render: {render * e / 1e6:.2f} muj")
print(f"Total: {(preprocess + radix_sort + render) * e / 1e6:.2f} muj")


# Test image descriptor match algorithms

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def visualize_matches(img1, img2, kp1, kp2, matches, title="Feature Matches"):
    """
    Visualize feature matching results
    :param img1: first input image
    :param img2: second input image
    :param kp1: keypoints from first image
    :param kp2: keypoints from second image
    :param matches: list of matched keypoints
    :param title: title for the plot
    """
    # Draw matches between images
    img_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches, None, 
                                 flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    
    # Convert to RGB format for matplotlib display
    img_matches_rgb = cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB)
    
    # Display the image
    plt.figure(figsize=(15, 10))
    plt.imshow(img_matches_rgb)
    plt.title(title)
    plt.axis('off')
    plt.show()

def match_features(img1, img2, method='ORB', ratio_thresh=0.7):
    """
    Main feature matching function
    :param img1: first input image
    :param img2: second input image
    :param method: feature matching method to use (SIFT, ORB, AKAZE, BRISK)
    :param ratio_thresh: ratio threshold for nearest neighbor matching
    :return: tuple of (keypoints1, keypoints2, good_matches)
    """
    # Convert images to grayscale
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    # Initialize feature detector and descriptor based on selected method
    if method == 'SIFT':
        detector = cv2.SIFT_create()
        norm_type = cv2.NORM_L2  # SIFT uses L2 norm for descriptor matching
    elif method == 'ORB':
        detector = cv2.ORB_create()
        norm_type = cv2.NORM_HAMMING  # ORB uses Hamming distance
    elif method == 'AKAZE':
        detector = cv2.AKAZE_create()
        norm_type = cv2.NORM_HAMMING
    elif method == 'BRISK':
        detector = cv2.BRISK_create()
        norm_type = cv2.NORM_HAMMING
    else:
        raise ValueError("Unsupported method. Choose from: SIFT, SURF, ORB, AKAZE, BRISK")
    
    # Detect keypoints and compute descriptors
    kp1, des1 = detector.detectAndCompute(gray1, None)
    kp2, des2 = detector.detectAndCompute(gray2, None)
    
    # Use brute-force matcher to find matches
    matcher = cv2.BFMatcher(norm_type)
    matches = matcher.knnMatch(des1, des2, k=2)  # Get top 2 matches for each descriptor
    
    # Apply ratio test to filter good matches
    good_matches = []
    for m, n in matches:
        if m.distance < ratio_thresh * n.distance:
            good_matches.append(m)
    
    return kp1, kp2, good_matches

# Read input images
base = "/home/xthuang/code/slam/copy/SplaTAM-Maxime/data/Replica/room0/results"
img1 = cv2.imread(base + '/frame000042.jpg')  # Replace with your image path
img2 = cv2.imread(base + '/frame000043.jpg')  # Replace with your image path

if img1 is None or img2 is None:
    raise ValueError("Failed to load images. Please check file paths")

# Available feature matching methods to test
methods = ['ORB', 'SIFT', 'SURF', 'AKAZE', 'BRISK']

for method in methods:
    try:
        print(f"Performing feature matching using {method}...")
        kp1, kp2, good_matches = match_features(img1, img2, method=method)
        print(f"Found {len(good_matches)} good matches")
        
        # Visualize the matching results
        visualize_matches(img1, img2, kp1, kp2, good_matches, 
                        title=f"Feature Matching using {method} (Matches: {len(good_matches)})")
    except Exception as e:
        print(f"Error with {method}: {str(e)}")